#1. O output desse processamento deve ser o dataset final reunindo todos os dados

##1.1 import apenas as bbibliotecas

In [1]:
import pandas as pd
import xarray as xr

##1.2 import todos os df de salinidade e temperatura e o df de batimetria

In [2]:
df_Salinidade = pd.read_parquet('/content/drive/MyDrive/2003-2020-Completo')
df_Batimetria = pd.read_parquet('/content/drive/MyDrive/batimetria_parquet')

##1.3 Criar um novo dataFrame com os dados de salinidade e temperatura

In [3]:
df = df_Salinidade.reset_index()
df_Batimetria = df_Batimetria.reset_index()

In [4]:
df.head(2)

,level_0,index,ANO,MES,DIA,LATITUDE,LONGITUDE,SALINIDADE,TEMPERATURA
0,0,0,2003,1,1,-0.01,0.00,34.745,301.532990
1,1,1,2003,1,1,-0.01,0.01,34.750,301.527008


##1.4 Inserir nesse novo df os dados de batimetria correspondentes as coordenadas

###1.4.1 o pandas possui um tempo muito grande para fazer manipulção e pesquisa, então vou usar o xarray multindex

In [5]:
batimetria_mtI = pd.DataFrame({'Longitude':df_Batimetria['Longitude'],'Latitude':df_Batimetria['Latitude'],'Batimetria':df_Batimetria['Batimetria']})
batimetria_mtI = batimetria_mtI.set_index(['Latitude','Longitude'])
batimetria_mtI = batimetria_mtI.to_xarray()

###1.4.2 agora com o multindex o tempo de execução cai bastante, no próximo bloco vou apenas pegar todas as batimetrias para as latitudes e longitudes

In [13]:
qtd = len(df)
bat = list()
count = 0
for bt in range(qtd):
  lat = df.loc[bt,'LATITUDE']
  lon = df.loc[bt,'LONGITUDE']
  try:
    batime = batimetria_mtI.sel(Latitude=[lat],Longitude=[lon]).Batimetria.data[0][0]
  except:
    batime = 0
  bat.append([batime])
  count = count + 1

In [25]:
qtd = len(bat)
batimetria_total = list()
for x in range(qtd):
  batimetria_total.append(bat[x][0])

##1.5 agora crio uma coluna no pandas para a batimetria e um novo df para ficar mais organizado

In [26]:
df['BATIMETRIA']=batimetria_total
df2 = df[['ANO','MES','DIA','LATITUDE','LONGITUDE','SALINIDADE','TEMPERATURA','BATIMETRIA']]

##1.6 existe um problema dentro da batimetria, muitas regiões estão com latitude e longitude unica de 1 metro, o que configura que quando se passa para 1km ela pode ficar mais fácil de gerar, mas em compensação existe uma relativa perda de dados. como não temos informações circulares do exato ponto da resolução em km para se tirar uma média, apenas deletei os valores em branco ou nao encontrados e salvei o dataSet final pronto para o turing de parametros

In [36]:
df2 = df2.loc[df2['BATIMETRIA']!= 0.0]
df2.to_parquet('/content/drive/MyDrive/dataSetFinal')